In [1]:
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')
import torch
torch.set_printoptions(linewidth=200)
import seaborn as sns
sns.set_style("darkgrid")

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))

# Results loader object does everything
from ionpy.analysis import ResultsLoader
from pathlib import Path
log_root = Path("/storage/vbutoi/scratch/ESE")
code_root = Path("/storage/vbutoi/projects/ESE")
rs = ResultsLoader()


# For using code without restarting.
%load_ext autoreload
%autoreload 
# For using yaml configs.
%load_ext yamlmagic

In [2]:
%%yaml inference_config

experiment:
    exp_root: '?'
    seed: 42

data:
    splits: ('val', 'cal')
    preload: False 

dataloader:
    batch_size: 1 
    num_workers: 0
    pin_memory: True 

log:
    root: '?'
    min_fg_pixels: '?' 
    gether_inference_stats: '?'
    summary_compute_global_metrics: '?' 
    log_interval: 10 
    show_examples: False 
    log_image_stats: True 
    log_pixel_stats: True
    track_ensemble_member_scores: True

<IPython.core.display.Javascript object>

In [3]:
%%yaml model_cfg

model:
    calibrator: '?'
    checkpoint: '?' 
    pretrained_exp_root : '?' 
    pretrained_select_metric: "val-dice_score"

<IPython.core.display.Javascript object>

In [4]:
%%yaml ensemble_cfg

ensemble:
    combine_fn: '?'
    combine_quantity: '?'
    member_w_metric: 'None'

<IPython.core.display.Javascript object>

In [5]:
import itertools
from typing import List, Optional
# ESE imports
from ese.scripts.utils import gather_exp_paths
from ese.experiment.models.utils import get_calibrator_cls


def get_ese_inference_configs(
    group_dict: dict,
    calibrators_list: List[str], 
    do_ensemble: bool = False, 
    log_image_stats: bool = True,
    log_pixel_stats: bool = True,
    ensemble_upper_bound: bool = False,
    normalize_opts: Optional[List[bool]] = [False],
    cal_stats_splits: Optional[List[str]] = [None],
    additional_args: Optional[dict] = None,
):
    # For ensembles, we have three choices for combining the predictions.
    ens_cfg_options=[
        ('mean', 'logits'), 
        ('mean', 'probs'), 
        ('product', 'probs')
    ]
    # Keep a list of all the run configuration options.
    calibrator_option_list = []
    # Gather the different config options.
    run_cfg_options = list(itertools.product(
        calibrators_list, 
        ens_cfg_options, 
        normalize_opts, 
        cal_stats_splits
    ))
    # Using itertools, get the different combos of calibrators_list ens_cfg_options and ens_w_metric_list.
    for (calibrator, ens_cfg, do_norm, cal_stats_split) in run_cfg_options: 
        # Set a few things that will be consistent for all runs.
        ##################################################
        exp_root = log_root / "inference" / group_dict['exp_group']
        use_uncalibrated_models = (calibrator == "Uncalibrated") or ("Binning" in calibrator)
        # Define the set of default config options.
        default_config_options = {
            'experiment.exp_root': [str(exp_root)],
            'experiment.dataset_name': [group_dict['dataset']],
            'model.checkpoint': ["max-val-dice_score" if use_uncalibrated_models else "min-val-ece_loss"],
            'model.calibrator': [calibrator],
            'model.calibrator_cls': [get_calibrator_cls(calibrator)],
            'data.preload': [group_dict['preload']],
            'log.log_image_stats': [log_image_stats],
            'log.log_pixel_stats': [log_pixel_stats],
        }
        # Add the unique arguments for the binning calibrator.
        if "Binning" in calibrator:
            default_config_options['model.normalize'] = [do_norm]
            default_config_options['model.cal_stats_split'] = [cal_stats_split]
        # If additional args are provided, update the default config options.
        if additional_args is not None:
            default_config_options.update(additional_args)

        # Define where we get the base models from.
        if use_uncalibrated_models:
            inf_group_dir = log_root / "training" / group_dict['base_models_group']
        else:
            inf_group_dir = log_root / "calibration" / group_dict['calibrated_models_group'] / f"Individual_{calibrator}"

        # If you want to run inference on ensembles, use this.
        if do_ensemble:
            # Define where we want to save the results.
            if ensemble_upper_bound:
                log_root = exp_root / f"ensemble_upper_bounds"
            else:
                log_root = exp_root / f"{group_dict['dataset']}_Ensemble_{calibrator}"

            # Define where the set of base models come from.
            advanced_args = {
                'log.root': [str(log_root)],
                'model.pretrained_exp_root': [str(inf_group_dir)],
                'model.ensemble': [True],
                'ensemble.combine_fn': [ens_cfg[0]],
                'ensemble.combine_quantity': [ens_cfg[1]],
            }
        # If you want to run inference on individual networks, use this.
        else:
            advanced_args = {
                'log.root': [str(exp_root / f"{group_dict['dataset']}_Individual_{calibrator}")],
                'model.pretrained_exp_root': gather_exp_paths(str(inf_group_dir)), # Note this is a list of train exp paths.
                'model.ensemble': [False],
            }
        # Combine the default and advanced arguments.
        default_config_options.update(advanced_args)
        # Append these to the list of configs and roots.
        calibrator_option_list.append(default_config_options)

    # Return the list of different configs.
    return calibrator_option_list

In [6]:
exp_name = '02_14_24_WMH_AbsoluteHistograms' 
group_dict = {
    "dataset": "WMH",
    "exp_group": exp_name,
    "base_models_group": "01_08_24_WMH_Ensemble",
    "calibrated_models_group": "01_19_24_WMH_Foreground_Calibrators",
    "preload": False
}

# exp_name = '02_11_24_CityScapes_Inference' 
# group_dict = {
#     "dataset": "CityScapes",
#     "exp_group": exp_name,
#     "base_models_group": "01_25_24_CityScapes_Dice",
#     "calibrated_models_group": "01_26_24_CityScapes_Ensemble",
#     "preload": False
# }

In [7]:
additional_args = {
    "log.gether_inference_stats": [True],
    "log.summary_compute_global_metrics": [True]
}

### Standard Calibrators.

In [8]:
# do_ensemble = False 
# # Get the configs for the different runs.
# dataset_options = get_ese_inference_configs(
#     group_dict=group_dict,
#     calibrators_list=[
#         'Uncalibrated',
#         'Vanilla',
#         'TempScaling',
#         'VectorScaling',
#         'DirichletScaling',
#         'LTS',
#         'NectarScaling',
#     ], 
#     do_ensemble=do_ensemble, 
#     additional_args=additional_args
# )

### Binning Calibrators

In [9]:
do_ensemble = False 
# Get the configs for the different runs.
dataset_options = get_ese_inference_configs(
    group_dict=group_dict,
    calibrators_list=[
        # 'HistogramBinning',
        'NectarBinning',
    ], 
    do_ensemble=do_ensemble, 
    normalize_opts=[
        # True, 
        False
    ],
    cal_stats_splits=[
        'val',
        'cal'
    ],
    additional_args=additional_args
)

### Upper Bounds

In [10]:
# # # Get the configs for computing the upepr bounds

# dataset_options = get_ese_inference_configs(
#     group_dict=group_dict,
#     do_ensemble=True, 
#     ensemble_upper_bound=True,
#     calibrators_list=[
#         'Uncalibrated',
#     ], 
#     ens_cfg_options=[
#         ('upperbound', 'probs')
#     ]
# )

In [11]:
from ese.scripts.utils import get_option_product
from ionpy.util import Config
import yaml

# Load the inference cfg from local.
##################################################
cfg_root = code_root / "ese" / "experiment" / "configs" / "inference"
##################################################
with open(cfg_root / f"{group_dict['dataset']}.yaml", 'r') as file:
    dataset_inference_cfg = yaml.safe_load(file)
with open(cfg_root / "Calibration_Metrics.yaml", 'r') as file:
    cal_metrics_cfg = yaml.safe_load(file)
##################################################
base_cfg = Config(inference_config).update([model_cfg, dataset_inference_cfg, cal_metrics_cfg])
if do_ensemble:
    base_cfg = base_cfg.update(ensemble_cfg)

# Get the configs
cfgs = get_option_product(exp_name, dataset_options, base_cfg)

In [12]:
len(cfgs)

24

## Running Jobs

In [13]:
from ese.experiment.analysis.run_inference import get_cal_stats

/local/vbutoi/envs/UniverSegTF/lib/python3.9/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_outputs" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [15]:
from ese.experiment.experiment import run_ese_exp

###### Run individual jobs
run_ese_exp(
    config=cfgs[0], 
    job_func=get_cal_stats,
    run_name='debug',
    show_examples=False,
    gpu='3',
) 

Set seed: 42


/storage/vbutoi/projects/ionpy/util/libcheck.py:49: UserWarning: Intel MKL extensions not available for NumPy
  warn("Intel MKL extensions not available for NumPy")
/storage/vbutoi/projects/ionpy/util/libcheck.py:57: UserWarning: Using slow Pillow instead of Pillow-SIMD
  warn("Using slow Pillow instead of Pillow-SIMD")


Set seed: 43


TypeError: __init__() missing 1 required positional argument: 'discretized_neighbors'

In [ ]:
# from ese.experiment.experiment import submit_ese_exps 
# #### Run Batch Jobs
# submit_ese_exps(
#     config_list=cfgs,
#     job_func=get_cal_stats,
#     available_gpus=['0', '1', '2', '3']
#     # available_gpus=['4', '5', '6', '7']
# )

## Here we run upper-bounds.

In [ ]:
from ese.experiment.analysis.ensemble_upperbound import get_ensemble_ub

In [ ]:
# ###### Run individual jobs
# run_ese_exp(
#     config=cfgs[0], 
#     job_func=get_ensemble_ub,
#     run_name='debug',
#     gpu='0',
# ) 

In [ ]:
# ###### Run Batch Jobs
# submit_ese_exps(
#     config_list=cfgs,
#     job_func=get_ensemble_ub,
#     available_gpus=['1']
# )